# C++ Cling Examples for STM32F103C8T6 Blue Pill

Load REMCU shared libray

In [ ]:
.L libremcu

Add path with header files

In [ ]:
.I remcu_include/

Including necessary header files.
The “remcu.h” header must be always included before any MCU header files.


In [ ]:
#include "remcu.h"
#include "stm32f10x.h"

Connection to a debug server. The [REMCU](https://remotemcu.github.io/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/). There is using the “remcu_connect2OpenOCD” function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

Before that, plug stm32 board with debugger to PC
![Image](img/connection.png)
run openocd utility in command line:
```
openocd -f interface/stlink-v2.cfg -f target/stm32f1x.cfg
```
Result:
![Image](img/openocd_unix.png)

In [ ]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const uint16_t default_gdb_port = 3333; // if you use remcu_connect2GDB function
const int timeout_sec = 3; // It can not be negative
bool connection = false;

In [ ]:
connection = remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server  etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [ ]:
//const uint16_t default_openocd_gdb_port = 3333;
//const uint16_t default_jlink_port = 2331;
//const uint16_t default_stlink_port = 61234;
//const uint16_t custom_port = ...;
//connection = remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port /*default_jlink_port*/ /*default_stlink_port*/, timeout_sec)

In [ ]:
if(connection == false){
    fprintf(stderr, "1. Check connection to debug server\n");
    fprintf(stderr, "2. Check debug server running\n");
    fprintf(stderr, "3. Check the debugged MCU is correct\n");
    exit(1);
}

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page) 
    
If connection were success... Jupyter Output:
![Image](img/success_connection_for_unix.png)

Reset the MCU:

In [ ]:
remcu_resetRemoteUnit(__HALT)

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (startup_stm32f10x.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/system_stm32f10x.c file

In [ ]:
SystemInit();

### GPIO Example
![Image](img/gpio.png)

>The code below have been taken from **IO_Toggle** ([main.c](STM32F10x_StdPeriph_Lib_V3.5.0/Project/STM32F10x_StdPeriph_Examples/GPIO/IOToggle/main.c)) example of [Standard Peripheral Library(SPL) for STM32F10x line](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-libraries/stsw-stm32054.html)

GPIOD Periph clock enable:

In [ ]:
RCC_APB2PeriphClockCmd(RCC_APB2Periph_GPIOC, ENABLE);

Configure PC13 in output pushpull mode:

In [ ]:
GPIO_InitTypeDef GPIO_InitStructure;

GPIO_InitStructure.GPIO_Pin = GPIO_Pin_13;
GPIO_InitStructure.GPIO_Speed = GPIO_Speed_10MHz;
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_Out_PP;
GPIO_Init(GPIOC, &GPIO_InitStructure);

Flash led on:

In [ ]:
GPIO_SetBits(GPIOC, GPIO_Pin_13);

Flash led off:

In [ ]:
GPIO_ResetBits(GPIOC, GPIO_Pin_13)

If it didn’t happen, you can use the utility functions to detect errors...

### Utility Functions

The ***remcu_getErrorCount***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function returns the number of errors that occurred after connection.

In [ ]:
remcu_getErrorCount()

If no error occurred, result ***0***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

The ***remcu_isConnected***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function indicates status of the current debug server connection:

In [ ]:
remcu_isConnected()

Result: connected ***true*** , disconnected ***false***

Some of debug adapters can have hardware problems. The ***remcu_debuggerTest***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function enables to detect such adapters. The function returns a null pointer if the test is successful:


In [ ]:
remcu_debuggerTest()

If no error occurred, result ***nullptr***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

>If bugs are not discovered and the chip don't responds to driver function call or the result of work was not what you expected, see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

### ADC example
![adc_head](img/adc_head.png )
>The code below is based on **AnalogWatchdog** ([main.c](STM32F10x_StdPeriph_Lib_V3.5.0/Project/STM32F10x_StdPeriph_Examples/ADC/AnalogWatchdog/main.c)) example of [Standard Peripheral Library(SPL) for STM32F10x line](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-libraries/stsw-stm32054.html)

clock for ADC:

In [ ]:
RCC_ADCCLKConfig(RCC_PCLK2_Div4);

enable ADC system clock:

In [ ]:
RCC_APB2PeriphClockCmd(RCC_APB2Periph_ADC1, ENABLE);

GPIOA Periph clock enable:

In [ ]:
RCC_APB2PeriphClockCmd(RCC_APB2Periph_GPIOA, ENABLE);

Configure PA.04 (ADC Channel4) as analog input:

In [ ]:
GPIO_InitStructure.GPIO_Pin = GPIO_Pin_4;
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_AIN;
GPIO_Init(GPIOA, &GPIO_InitStructure);

define ADC base config:

In [ ]:
ADC_InitTypeDef ADC_InitStructure;

In [ ]:
ADC_InitStructure.ADC_Mode = ADC_Mode_Independent;
ADC_InitStructure.ADC_ScanConvMode = DISABLE;
ADC_InitStructure.ADC_ContinuousConvMode = DISABLE;
ADC_InitStructure.ADC_ExternalTrigConv = ADC_ExternalTrigConv_None;
ADC_InitStructure.ADC_DataAlign = ADC_DataAlign_Right;
ADC_InitStructure.ADC_NbrOfChannel = 1;
ADC_Init(ADC1, &ADC_InitStructure);

ADC1 regular channel4(PA4 pin) configuration:

In [ ]:
ADC_RegularChannelConfig(ADC1, ADC_Channel_4, 1, ADC_SampleTime_55Cycles5);

Enable ADC1:

In [ ]:
ADC_Cmd(ADC1, ENABLE);

Enable ADC1 reset calibration register:

In [ ]:
ADC_ResetCalibration(ADC1);

Check the end of ADC1 reset calibration register:

In [ ]:
while(ADC_GetResetCalibrationStatus(ADC1));

Start ADC1 calibration:

In [ ]:
ADC_StartCalibration(ADC1);

Check the end of ADC1 calibration:

In [ ]:
while(ADC_GetCalibrationStatus(ADC1));

Start ADC1 Software Conversion:

In [ ]:
ADC_SoftwareStartConvCmd(ADC1, ENABLE);

The function definition. This function does conversion and return the raw ADC value:

In [ ]:
uint16_t ADC_value(){
    ADC_SoftwareStartConvCmd(ADC1, ENABLE); // start conversion
    return ADC_GetConversionValue(ADC1);
}

ADC_value()

Prepare canvas, which shows ADC data. [Template](https://github.com/QuantStack/xplot/blob/3399c4b96dffb1fec9d9c44de74d3d223bb04789/notebooks/xplot.ipynb)
> First install [xplot](https://github.com/QuantStack/xplot) or make sure you have it
<details>
  <summary>The kernel of notebook should be  <b>C++14</b> (clck here)</summary>
    
![Image](img/Cpp14.png)  

</details>

In [ ]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

const size_t ADC_BUFF_SIZE = 100;
xpl::figure fig;
xpl::linear_scale sx, sy;
xpl::lines line(sx, sy);
std::vector<uint16_t> adc_data(ADC_BUFF_SIZE);
std::vector<int> x_line(ADC_BUFF_SIZE);
int i = 0;
for( auto & x : x_line){x = i++;}
line.x = x_line;
line.y = adc_data;
fig.add_mark(line);
    
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);
fig

Plug a potentiometer into the PA4 pin and run the code below. Rotate the shaft and you will get the voltage versus time chart:
<details>
  <summary> show 
<b>(click here) </b></summary>

![potentiometer](img/pot.png)
![var_resistor_scheme](img/var_resistor_scheme.png)
</details>

In [ ]:
#include <unistd.h>

for(size_t i = 0; i < 100; i++){
    adc_data[i % ADC_BUFF_SIZE] = ADC_value();
    //plot ADC data
    line.y = adc_data;
    usleep(200000);
}

<details>
  <summary>Result<b>(click to show)</b></summary>

![Image](img/adc_cpp.png)
</details>

<details>
  <summary>process
	<b>(click here) </b></summary>

![connection](img/f103_adc.gif)
</details>

Finalizing work with REMCU Library:

In [ ]:
remcu_disconnect()